In [55]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tqdm

status_page_link = 'https://glottolog.org/langdoc/status/browser?macroarea=&focus=sdt#2/21.0/149.8'
status_data = pd.read_csv('../docs/master_merge.csv')[['iso6393', 'glottocode']]

In [56]:
def get_links(): 
    link_base = 'https://glottolog.org/resource/languoid/id/'
    ids = pd.read_csv('../docs/master_merge.csv')['glottocode'].values

    links = []
    for i in tqdm.tqdm(ids):
        links.append(link_base + i)
    return links

links = get_links()
    

100%|██████████| 7590/7590 [00:00<00:00, 1177669.70it/s]


In [57]:
from tqdm import tqdm
import numpy as py
# FILEPATH: /Users/gracewarren/LanguageStatusVisualizationProject/ipynb_files/glottolog_scraper.ipynb
# new_data = pd.DataFrame(columns = ['glottocode', 'aes_status'])

for link in tqdm(links):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')

    aes_status = soup.find('dt', text='AES status:')
    if aes_status is not None:
        status_data.loc[status_data['glottocode'] == link.split('/')[-1], 'aes_status'] = aes_status.find_next_sibling('dd').string
    else:
        status_data.loc[status_data['glottocode'] == link.split('/')[-1], 'aes_status'] = 'N/A'
        
    wiki_link = soup.find('a', title='Wikipedia')
    # print(wiki_link['href'])
    if wiki_link is not None:
        
        status_data.loc[status_data['glottocode'] == link.split('/')[-1], 'Wikipedia_Url'] = wiki_link['href']
    else:
        status_data.loc[status_data['glottocode'] == link.split('/')[-1], 'Wikipedia_Url'] = ''

status_data.to_csv('glottolog_data.csv', index=False)


  0%|          | 0/7590 [00:00<?, ?it/s]

/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_21766/589282450.py:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  aes_status = soup.find('dt', text='AES status:')
  0%|          | 1/7590 [00:01<3:34:20,  1.69s/it]/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_21766/589282450.py:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  aes_status = soup.find('dt', text='AES status:')
100%|██████████| 7590/7590 [2:17:06<00:00,  1.08s/it]  


In [49]:
# Find link to Wikipedia page
soup.find_all('a')
soup.find('a', title='Wikipedia')['href']

'https://en.wikipedia.org/wiki/Ankave_language'

In [33]:
status_data.loc[status_data['glottocode'] == link.split('/')[-1]]

,iso6393,glottocode,aes_status,Wikipedia_Url
1,aab,alum1246,not endangered,NaN


In [35]:
status_data.loc[status_data['glottocode'] == link.split('/')[-1], 'Wikipedia_Url'] = 0

In [36]:
status_data.loc[status_data['glottocode'] == link.split('/')[-1], 'Wikipedia_Url']

1    0
Name: Wikipedia_Url, dtype: object

In [81]:
status_data.to_csv('../csv_files/status_data.csv')

In [82]:
merged = pd.read_csv('../docs/master_merge.csv')

In [92]:
merge_new = pd.merge(merged, status_data, left_on = 'glottocode', right_on = 'glottocode', how = 'left')
merge_new['iso6393'] = merge_new.apply(lambda x: x['iso6393_x'] if pd.isnull(x['iso6393_y']) else x['iso6393_y'], axis = 1)
merge_new = merge_new.drop(columns = ['iso6393_x', 'iso6393_y'])
merge_new

,Unnamed: 0,name_x,type,scope,family,dialects,regions,off_lang,rec_min_lang,url,Degree of endangerment,speakers,name_y,level,macroarea,latitude,longitude,glottocode,aes_status,iso6393
0,0,Ghotuo,living,individual,<ul><li>Uknown</li></ul>,Uknown,Data unavailable,Unknown,Unknown,Data unavailable,Data unavailable,Uknown,Ghotuo,language,Africa,7.115510,5.956630,ghot1243,not endangered,aaa
1,1,Alumu-Tesu,living,individual,<ul><li>Uknown</li></ul>,Uknown,Data unavailable,Unknown,Unknown,Data unavailable,Data unavailable,Uknown,Arum,language,Africa,9.015130,8.559560,alum1246,not endangered,aab
2,2,Ari,living,individual,<ul><li>Uknown</li></ul>,Uknown,Data unavailable,Unknown,Unknown,Data unavailable,Severely endangered,80.0,Ari,language,Papunesia,-7.955260,142.400000,arii1243,moribund,aac
3,3,Amal,living,individual,<ul><li>Uknown</li></ul>,Uknown,Data unavailable,Unknown,Unknown,Data unavailable,Data unavailable,Uknown,Amal,language,Papunesia,-4.045170,141.995000,amal1242,shifting,aad
4,4,Arbëreshë Albanian,living,individual,<ul><li>Uknown</li></ul>,Uknown,Data unavailable,Unknown,Unknown,Data unavailable,Data unavailable,Uknown,Arbëreshë Albanian,language,Eurasia,38.911000,16.716400,arbe1236,threatened,aae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7585,7863,Yang Zhuang,living,individual,<ul><li>Uknown</li></ul>,Uknown,Data unavailable,Unknown,Unknown,Data unavailable,Data unavailable,Uknown,Yang Zhuang,language,Eurasia,23.091864,106.393188,yang1286,not endangered,zyg
7586,7864,Youjiang Zhuang,living,individual,<ul><li>Uknown</li></ul>,Uknown,Data unavailable,Unknown,Unknown,Data unavailable,Data unavailable,Uknown,Youjiang Zhuang,language,Eurasia,23.750000,106.900000,youj1238,not endangered,zyj
7587,7865,Yongnan Zhuang,living,individual,<ul><li>Uknown</li></ul>,Uknown,Data unavailable,Unknown,Unknown,Data unavailable,Data unavailable,Uknown,Yongnan Zhuang,language,Eurasia,22.800000,108.000000,yong1275,not endangered,zyn
7588,7866,Zyphe Chin,living,individual,<ul><li>Uknown</li></ul>,Uknown,Data unavailable,Unknown,Unknown,Data unavailable,Data unavailable,Uknown,Zyphe,language,Eurasia,22.524000,93.264000,zyph1238,not endangered,zyp


In [93]:
merge_new['name'] = merge_new.apply(lambda x: x['name_x'] if pd.isnull(x['name_y']) else x['name_y'], axis = 1)
merge_new = merge_new.drop(columns = ['name_x', 'name_y', 'Unnamed: 0'])
merge_new = merge_new[['name', 'iso6393', 'glottocode', 'Degree of endangerment', 'aes_status', 'level', 'macroarea', 'type', 'scope', 'speakers', 'family', 'dialects', 'regions', 'off_lang', 'rec_min_lang', 'latitude', 'longitude', 'url']]

In [94]:
merge_new.to_csv('../docs/master_merge.csv')

In [95]:
merge_new['aes_status'].unique()

array(['not endangered', 'moribund', 'shifting', 'threatened', 'extinct',
       'nearly extinct', 'N/A'], dtype=object)